In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
print('Enter the comapies for portfolio')
portfolio = {}
companies = []
while True:
    company = input('Enter the company name: ')
    if company == '':
        break
    companies.append(company)

for company in companies:
    number_of_shares = int(input(f'Enter the number of shares for {company}: '))
    portfolio[company] = number_of_shares


Enter the comapies for portfolio


In [9]:
portfolio = {
    'TCS' : 20,
    'HDFCBANK' : 3000,
    'RELIANCE' : 234,
    'INFY' : 354,
}

In [10]:
total_value_of_each_company  = {}
for company, quantity in portfolio.items():
    try:
        url = f'https://www.screener.in/company/{company}/'
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content, 'html.parser')
        table = soup.find_all('table')
        data = pd.read_html(str(table))[1].to_dict()
        net_profit_list = []
        for i in range(1, 13):
            net_profit_list.append(int(data[list(data)[i]][9]))
        company_id = int(soup.find('div', {'id': 'company-info'})['data-company-id'])
        compounded_profit_growth = net_profit_list[-1] / net_profit_list[0]
        compounded_profit_growth = compounded_profit_growth ** (1/len(net_profit_list)) - 1
        
        resp = requests.get(f'https://www.screener.in/api/company/{company_id}/chart/?q=Price-DMA50-DMA200-Volume&days=10000&consolidated=true')
        data = resp.json()
        price_data = data['datasets'][0]
        price_list = []
        price_date_list = []
        for i in price_data['values']:
            price_list.append(float(i[1]))
            price_date_list.append((i[0]))
        price = price_list[-1]
        total_value_of_each_company[company] = ((price * quantity), compounded_profit_growth*100)
        print(f"Value of {company} is {price * quantity}")
    except:
        print(f"Error fetching data for {company}")



C:\Users\Ishant\AppData\Local\Temp\ipykernel_26212\3242283025.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[1].to_dict()


Value of TCS is 91030.0


C:\Users\Ishant\AppData\Local\Temp\ipykernel_26212\3242283025.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[1].to_dict()


Value of HDFCBANK is 4877400.0


C:\Users\Ishant\AppData\Local\Temp\ipykernel_26212\3242283025.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[1].to_dict()


Value of RELIANCE is 701379.9


C:\Users\Ishant\AppData\Local\Temp\ipykernel_26212\3242283025.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[1].to_dict()


Value of INFY is 662935.8


In [11]:

total_value_of_portfolio = 0
for i in total_value_of_each_company:
    total_value_of_portfolio += total_value_of_each_company[i][0]


percent_contribution_growth = []
holding_contribution = []

for company in total_value_of_each_company:
    # print(f"Percentage of {company} is {total_value_of_each_company[company][0] / total_value_of_portfolio * 100:.2f}%")
    percent_holding, compounded_profit_growth = total_value_of_each_company[company]
    growth_percent = percent_holding / total_value_of_portfolio * compounded_profit_growth
    percent_contribution_growth.append(growth_percent)
    holding_contribution.append(percent_holding / total_value_of_portfolio * 100)
    # print(f"Holding Percentage of {company} is {percent_holding / total_value_of_portfolio * 100:.2f}%")
    # print(f"Growth contribution of {company} is {percent_holding / total_value_of_portfolio * compounded_profit_growth:.2f}%")

if len(portfolio) != len(holding_contribution):
    print("Fatal Error in calculation")

if len(holding_contribution) != len(percent_contribution_growth):
    print("Error in calculation")

for i in range(len(holding_contribution)):
    print(f"Contribution of {list(portfolio.keys())[i]} is {holding_contribution[i]:.2f}% and growth contribution is {percent_contribution_growth[i]:.2f}%")

print(f"Total value of portfolio is {total_value_of_portfolio}")
print(f"Total growth of portfolio is {sum(percent_contribution_growth):.2f}%")
print('This assumption is based on profit growth and increment in PE is not considered. User is advised to take decision based on their own research and analysis')

Contribution of TCS is 1.44% and growth contribution is 0.15%
Contribution of HDFCBANK is 77.02% and growth contribution is 15.51%
Contribution of RELIANCE is 11.08% and growth contribution is 0.66%
Contribution of INFY is 10.47% and growth contribution is 1.00%
Total value of portfolio is 6332745.7
Total growth of portfolio is 17.32%
This assumption is based on profit growth and increment in PE is not considered. User is advised to take decision based on their own research and analysis
